In [1]:
import numpy as np
import matplotlib.pyplot as pl
import dask.array as da

In [20]:
def radial_profile(data: np.ndarray, centre: tuple):
    """_summary_

    Parameters
    ----------
    data : np.ndarray
        _description_
    centre : tuple
        _description_

    Returns
    -------
    _type_
        _description_
    """
    
    x, y = np.indices((data.shape))
    r = np.sqrt((x - centre[0]) ** 2 + (y - centre[1]) ** 2)
    r = r.astype(int)
    tbin = np.bincount(np.ravel(r), np.ravel(data))
    nr = np.bincount(np.ravel(r))
    radialprofile = tbin / nr
    return radialprofile

In [94]:
#read in data
from ddm.data_handling import readLIF
dData = readLIF('../tests/data/testData3series.lif')

from ddm.data_handling import readND2
dData2 = readND2('../tests/data/testData10frames.nd2')

In [21]:
#dummy data for testing
dData = np.random.randint(0, 256, size = (20, 512, 512))

In [38]:
#method 1 - high q for A = 0 - rework to optimise the FFTs???
def fitABhighQ(dumData):
    sqFFT = 2*np.abs(np.fft.fft2(dumData))*np.abs(np.fft.fft2(dumData))
    sqFFT = np.fft.fftshift(sqFFT)
    sqFFTmean = np.mean(sqFFT, axis = 0)
    return sqFFTmean
    #sqFFTrad = radial_profile(sqFFTmean, (np.shape(dumData)[1]/2, np.shape(dumData)[2]/2))
    #b = np.mean(sqFFTrad[-100:-50]) #change depending on size of array
    #a = sqFFTrad - b
    #return a, b

In [95]:
dData
print(dData[0].data.compute())

TypeError: '<' not supported between instances of 'slice' and 'int'

In [77]:
dData2

<xarray.DataArray '_dask_block-c23fa2142aef5bd34412e61b2c424a51' (T: 10,
                                                                  Y: 1024,
                                                                  X: 1024)>
dask.array<_dask_block, shape=(10, 1024, 1024), dtype=uint16, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * Y        (Y) float64 0.0 0.065 0.13 0.195 0.26 ... 66.3 66.37 66.43 66.5
  * X        (X) float64 0.0 0.065 0.13 0.195 0.26 ... 66.3 66.37 66.43 66.5
  * T        (T) float64 0.0 20.0 40.01 60.01 80.01 ... 120.0 140.0 160.0 180.0
Attributes:
    xyScale:  0.065
    tScale:   20.003011067708332

In [78]:
meanSQ = fitABhighQ(dData.data)

In [62]:
meanSQ.compute()

TypeError: '<' not supported between instances of 'slice' and 'int'

In [ ]:
#method 2 - delay time is zero then A = real part of squared stuff
fTFt = np.fft.fftshift(2*np.fft.fft2(dData)*np.conj(np.fft.fft2(dData, axes = (0,1))))
aFull = np.real(np.mean(fTFt, axis = 0))
c = radial_profile(aFull, (512, 512))
pl.figure(0)
pl.plot(c[10:], color = 'r')
pl.plot(A[10:], color = 'b')

SqFFT = 2*np.abs(np.fft.fft2(dData)**2)
SqFFT = np.fft.fftshift(SqFFT)
SqFFTmean = np.mean(SqFFT, axis = 0)
SqFFTrad = radial_profile(SqFFTmean, (512, 512))

d = SqFFTrad - c

pl.figure(1)
pl.plot(d[10:], color = 'm')
pl.plot(sqFFTrad[10:], color = 'c')

In [ ]:
from ddm.processing import ddm

In [ ]:
result = np.array([None for i in range(10)])
for delay in range(10):
    result[delay] = ddm(dData, delay).compute()

In [ ]:
result2 = np.dstack(result)[0]

q = 200

f = 1.-(result2[q]-B)/A[q]

pl.figure(2)
pl.plot(f)
pl.xlabel('tau')